In [3]:
import pandas as pd
from datasets import load_dataset
import os
import random

def prepare_wtq_subset(sample_size=50):
    print(f">>> 正在从 Hugging Face 加载 WikiTableQuestions 数据集...")
    
    # 1. 加载数据集 (只加载测试集)
    # 这可能会花一点时间下载，取决于网速
    dataset = load_dataset("wikitablequestions", split="test")
    
    print(f"    原始测试集大小: {len(dataset)} 条")
    
    # 2. 随机采样
    # 为了保证实验可复现，设置随机种子
    random.seed(42)
    indices = random.sample(range(len(dataset)), sample_size)
    
    sampled_data = dataset.select(indices)
    
    output_dir = "benchmark_real"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    tasks = []
    
    print(f">>> 正在处理 {sample_size} 条样本，生成 CSV...")
    
    for i, item in enumerate(sampled_data):
        # WTQ 的数据结构: {'table': {'header': [...], 'rows': [...]}, 'question': '...'}
        
        # 提取表格
        header = item['table']['header']
        rows = item['table']['rows']
        
        # 创建 DataFrame
        df = pd.DataFrame(rows, columns=header)
        
        # 保存为独立的 CSV 文件 (模拟真实环境)
        csv_filename = os.path.join(output_dir, f"wtq_{i}.csv")
        df.to_csv(csv_filename, index=False)
        
        # 记录任务信息
        tasks.append({
            "id": item['id'],
            "file": csv_filename,
            "question": item['question'],
            "answers": item['answers'] # 标准答案（用于人工核对）
        })
    
    # 保存任务列表
    task_df = pd.DataFrame(tasks)
    task_df.to_csv("real_benchmark_tasks.csv", index=False)
    
    print(f"✅ 成功！已准备 {sample_size} 个真实表格任务。")
    print(f"    任务列表保存在: real_benchmark_tasks.csv")
    print(f"    表格文件保存在: {output_dir}/")

if __name__ == "__main__":
    prepare_wtq_subset(sample_size=20) # 先跑 20 个试试水，没问题了再改成 100

ImportError: cannot import name 'xopen' from 'datasets.utils.file_utils' (C:\Users\world\AppData\Roaming\Python\Python312\site-packages\datasets\utils\file_utils.py)